# LOAD DATA AND PREPROCESSING OF SEED V

For more information about ```mne```, please go to website: [https://mne.tools/stable/index.html](https://mne.tools/stable/index.html)

In [27]:
#Library Processing EEG
import mne

#Library Data Analisys
import pandas as pd
import numpy as np

#Library Tools System
import os
from tqdm import tqdm
import re
import time
import shutil

# No Warning
import warnings
warnings.filterwarnings("ignore")
mne.set_log_level("ERROR")

print(np.__version__)
print(mne.__version__)

1.23.5
1.3.1


Using the cnt files containing the data from the experiments into MNE raw objects (a specialized library, among other things, for the treatment of EEG).
The content is filtered by removing all frequency components below 0.5 Hz and above 55 Hz because those components are considered noise.
Each trial corresponds to the reading of 62 signals (one per channel) while a subject watches a video labeled with an emotion (positive, negative, neutral, disgust and fear). We are going to use just the trial with positive, negative or neutral emotion.
Knowing that each of the MNE raw objects contains 62 signals in which the trials are stored one after another, it is necessary to divide these signals using the start and end times for each trial.

In [28]:
os.getcwd()

'C:\\Users\\macka\\TFM_WD\\ORI\\SEED_V\\EEG_raw\\EEG_raw\\S1'

In [29]:
# Get a list of the names of all the files in the folder

ruta='C:/Users/macka/TFM_WD/ORI/SEED_V/EEG_raw/EEG_raw/S1/'
archivos = os.listdir(ruta)
names=[]
# Appends in the names list just the cnt files
for archivo in archivos:
    if archivo.endswith(".cnt"):
        names.append(archivo)
        
os.chdir(ruta)

In [30]:
# To measure the execution time of this code
start_time = time.time()  # Record the start time

# Information about this Dataset (when a trial starts and when ends in each file, the label of the trial, etc
trial_num=['Trial-1','Trial-2','Trial-3','Trial-4','Trial-5','Trial-6','Trial-7','Trial-8','Trial-9','Trial-10',
            'Trial-11','Trial-12','Trial-13','Trial-14','Trial-15']
# #Session 1
emotions= ['POSITIVE', 'FEAR', 'NEUTRAL', 'NEGATIVE', 'DISGUST','POSITIVE', 'FEAR', 'NEUTRAL', 'NEGATIVE', 'DISGUST','POSITIVE', 'FEAR', 'NEUTRAL', 'NEGATIVE', 'DISGUST']
# #Session 2
# emotions= ['NEGATIVE', 'FEAR', 'NEUTRAL', 'DISGUST', 'POSITIVE','POSITIVE', 'DISGUST', 'NEUTRAL', 'NEGATIVE', 'FEAR','NEUTRAL', 'POSITIVE', 'FEAR', 'NEGATIVE', 'DISGUST']
# #Session 3
# emotions= ['NEGATIVE', 'FEAR', 'NEUTRAL', 'DISGUST', 'POSITIVE', 'POSITIVE','DISGUST', 'NEUTRAL', 'NEGATIVE', 'FEAR', 'NEUTRAL', 'POSITIVE','FEAR', 'NEGATIVE', 'DISGUST']

# First Sessions
start_seconds= [30, 132, 287, 555, 773, 982, 1271, 1628, 1730, 2025, 2227, 2435, 2667, 2932, 3204]
end_seconds= [102, 228, 524, 742, 920, 1240, 1568, 1697, 1994, 2166, 2401, 2607, 2901, 3172, 3359]
    #Session 2:
# start_seconds= [30, 299, 548, 646, 836, 1000, 1091, 1392, 1657, 1809, 1966, 2186, 2333, 2490, 2741]
# end_seconds= [267, 488, 614, 773, 967, 1059, 1331, 1622, 1777, 1908, 2153, 2302, 2428, 2709, 2817]
#     #Session 3=
# start_seconds= [30, 353, 478, 674, 825, 908, 1200, 1346, 1451, 1711, 2055, 2307, 2457, 2726, 2888]
# end_seconds= [321, 418, 643, 764, 877, 1147, 1284, 1418, 1679, 1996, 2275, 2425, 2664, 2857, 3066]

# # Creation of the folders where the new files will be stored
output_folder_positive = 'SEED_V/POSITIVE'
output_folder_neutral = 'SEED_V/NEUTRAL'
output_folder_negative = 'SEED_V/NEGATIVE'
output_folder_fear = 'SEED_V/FEAR'
output_folder_disgust = 'SEED_V/DISGUST'

if not os.path.exists(output_folder_positive):
    os.makedirs(output_folder_positive)
if not os.path.exists(output_folder_neutral):
    os.makedirs(output_folder_neutral)
if not os.path.exists(output_folder_negative):
    os.makedirs(output_folder_negative)
if not os.path.exists(output_folder_fear):
    os.makedirs(output_folder_fear)
if not os.path.exists(output_folder_disgust):
    os.makedirs(output_folder_disgust)

#Creation of the new files, separating the data of the original ones
for name in tqdm(names, desc="Processing", total=len(names),leave=False):
    
    # Data loading and elimination of non desired data
    eeg_raw = mne.io.read_raw_cnt(name, preload=True)
    useless_ch = ['M1', 'M2', 'VEO', 'HEO']
    eeg_raw.drop_channels(useless_ch)
    
    # Apply a band-pass filter to remove noise
    l_freq = 0.5  # low cut frequency
    h_freq = 55  # High cut frequency
    eeg_raw.filter(l_freq, h_freq)
    
    # Division of the EEG signal into diferent trial
    for trial, start, end, emotion in tqdm(zip(trial_num, start_seconds, end_seconds, emotions), desc=f"Processing {name}", total=len(trial_num),leave=False):

        # Extraction of the trial data
        start_sample = start * eeg_raw.info['sfreq']
        end_sample = end * eeg_raw.info['sfreq']
        eeg_trial = eeg_raw.copy().crop(tmin=start_sample / eeg_raw.info['sfreq'], tmax=end_sample / eeg_raw.info['sfreq'])

        # Saving the trial in a fif file
        file_basename = os.path.splitext(os.path.basename(name))[0]
        output_filename = f"{file_basename}_{trial}_{emotion}_start_{start}_end_{end}.fif"
        if emotion == 'POSITIVE':
            output_path = os.path.join(output_folder_positive, output_filename)
        elif emotion == 'NEUTRAL':
            output_path = os.path.join(output_folder_neutral, output_filename)
        elif emotion == 'NEGATIVE':
            output_path = os.path.join(output_folder_negative, output_filename)
        elif emotion == 'FEAR':
            output_path = os.path.join(output_folder_fear, output_filename)
        elif emotion == 'DISGUST':
            output_path = os.path.join(output_folder_disgust, output_filename)
        eeg_trial.save(output_path, overwrite=True)

elapsed_time = time.time() - start_time
print(f"The code execution lasted {elapsed_time:.2f}")


Processing:  75%|███████▌  | 12/16 [04:03<01:23, 20.91s/it]                 


Processing 9_1_20180724.cnt: 100%|██████████| 15/15 [00:09<00:00,  1.57it/s]
                                                                            

El código tardó 346.30 segundos en ejecutarse.


# Explicaciones 

**Noise Filters**:

**Components of the signal of interest**: Depending on the analysis objective and the data characteristics, you may want to preserve certain frequency bands in the signal. For example, typical frequency bands in EEG analysis include delta (0.5-4 Hz), theta (4-8 Hz), alpha (8-12 Hz), beta (12-30 Hz), and gamma (30-100 Hz). Make sure the cutoff frequencies do not remove the frequency bands of interest.

**Study Requirements**: Some studies may require analysis in specific frequency ranges. For example, in the study of brain activity related to emotions, alpha and beta frequency bands may be of particular interest. Adjust the cutoff frequencies according to the specific requirements of your study.